In [1]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import HEDdetector
from PIL import Image
import numpy as np
import os
import random
from IPython.display import display, HTML
import base64
from io import BytesIO

# 1. Setup Models and Processors
# Load the HED detector for creating scribble maps
annotator = HEDdetector.from_pretrained('lllyasviel/Annotators')

# Load the scribble ControlNet model
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_scribble", 
    torch_dtype=torch.float16
)

# Load the Stable Diffusion pipeline with the ControlNet
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", 
    controlnet=controlnet, 
    torch_dtype=torch.float16
)

# Use a faster scheduler
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# Move to GPU
pipe.to("cuda")

c:\Users\paul-\miniconda3\envs\py311-cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0918 11:18:15.322000 26164 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0918 11:18:15.322000 26164 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
c:\Users\paul-\miniconda3\envs\py311-cuda\Lib\site-packages\diffusers\models\transformers\transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Tra

StableDiffusionControlNetPipeline {
  "_class_name": "StableDiffusionControlNetPipeline",
  "_diffusers_version": "0.28.2",
  "_name_or_path": "runwayml/stable-diffusion-v1-5",
  "controlnet": [
    "diffusers",
    "ControlNetModel"
  ],
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "UniPCMultistepScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [2]:
prompt = "minimalistic sketch, simple and abstract drawing, white background, amateur drawing"
negative_prompt = "shadows, graphic, many details, gradients, filled areas"

In [3]:
import math
from pathlib import Path
from tqdm.auto import tqdm

# 2. Configuration for batch sketch generation
# Adjust BASE_PATH if notebook moved.
BASE_PATH = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
INPUT_FOLDER = (BASE_PATH / '../../output/amazing_logos_v4/images/balanced_sample_2k_512x512').resolve()
OUTPUT_FOLDER = (INPUT_FOLDER.parent / (INPUT_FOLDER.name + '_sketches')).resolve()

OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)
print(f'Input folder: {INPUT_FOLDER}')
print(f'Output folder: {OUTPUT_FOLDER}')

# Generation parameters
seed = 42  # Set to None for random
num_inference_steps = 20
cfg_scale = 7.0
eta = 0.0  # Not used by all schedulers

# Safety: some pipelines include a safety checker; disable if you want raw output
if hasattr(pipe, 'safety_checker'):
    pipe.safety_checker = lambda images, clip_input: (images, [False] * len(images))

# Helper: create control (scribble) image from source
# We use the HED detector to get edges; optionally post-process to binary/threshold

def make_control_image(pil_img: Image.Image, hed_processor: HEDdetector):
    np_img = np.array(pil_img.convert('RGB'))
    # HED returns a PIL image (white background, black lines) typically
    control = hed_processor(np_img)
    # Ensure size is 512x512 (expected by SD 1.5 usually)
    control = control.resize((512, 512))
    return control

# Collect images (png/jpg/jpeg)
valid_ext = {'.png', '.jpg', '.jpeg', '.webp'}
all_images = [p for p in INPUT_FOLDER.iterdir() if p.suffix.lower() in valid_ext]
print(f'Total images found: {len(all_images)}')

# Resume support: skip already processed files
existing = {p.stem for p in OUTPUT_FOLDER.glob('*.*')}
remaining = [p for p in all_images if p.stem not in existing]
print(f'Already done: {len(all_images) - len(remaining)} | Remaining: {len(remaining)}')

if seed is not None:
    generator = torch.Generator(device='cuda').manual_seed(seed)
else:
    generator = torch.Generator(device='cuda')

# Main loop
for img_path in tqdm(remaining, desc='Generating sketches'):
    try:
        orig = Image.open(img_path).convert('RGB')
        # Resize/crop to 512x512 if needed
        if orig.size != (512, 512):
            orig = orig.resize((512, 512), Image.LANCZOS)

        control_image = make_control_image(orig, annotator)

        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=control_image,
            num_inference_steps=num_inference_steps,
            guidance_scale=cfg_scale,
            generator=generator,
        )
        out_img = result.images[0]

        out_file = OUTPUT_FOLDER / f'{img_path.stem}.png'
        out_img.save(out_file)
    except Exception as e:
        print(f'Failed {img_path.name}: {e}')

print('Done.')

Input folder: C:\studium\master_thesis\data_prep\output\amazing_logos_v4\images\balanced_sample_2k_512x512
Output folder: C:\studium\master_thesis\data_prep\output\amazing_logos_v4\images\balanced_sample_2k_512x512_sketches
Total images found: 2000
Already done: 0 | Remaining: 2000


100%|██████████| 20/20 [00:01<00:00, 14.56it/s]0:00<?, ?it/s]

100%|██████████| 20/20 [00:01<00:00, 19.91it/s]0:02<1:09:20,  2.08s/it]

100%|██████████| 20/20 [00:01<00:00, 19.47it/s]0:03<55:16,  1.66s/it]  

100%|██████████| 20/20 [00:01<00:00, 19.67it/s]0:04<50:55,  1.53s/it]

100%|██████████| 20/20 [00:01<00:00, 19.91it/s]0:06<48:42,  1.46s/it]

100%|██████████| 20/20 [00:01<00:00, 19.73it/s]0:07<47:35,  1.43s/it]

100%|██████████| 20/20 [00:01<00:00, 19.73it/s]0:08<46:46,  1.41s/it]

100%|██████████| 20/20 [00:01<00:00, 19.90it/s]0:10<45:37,  1.37s/it]

100%|██████████| 20/20 [00:01<00:00, 19.45it/s]00:14<44:52,  1.35s/it]

100%|██████████| 20/20 [00:01<00:00, 19.60it/s]00:15<44:51,  1.35s/it]

100%|██████████| 20/20 [00:01<00:00, 19.82it/s]00:16<45:02,  1.36s/it]

100%|██████████| 20/20 [00:01<00:00, 19.66it/s]00:18<45:02,  1.36s/it]

100%|██████████| 20/20 [00:01<00:00, 19.69it/s]00:21<44:42,  1.35s/it]

100%|██████████| 20/20 [00:01<00:00, 19.37it/s]00:23<44:45,  1.35s/it]

100%

Done.
